# Loading and Inspecting XPCS Data

This notebook demonstrates how to work with XPCS correlation data:
loading, inspecting, filtering, and visualizing G2 autocorrelation functions
and SAXS profiles.

## Prerequisites

- xpcsviewer installed (`uv sync` or `pip install -e .`)

> **Note**: This notebook uses synthetic data so it runs without external HDF5 files.
> In practice, you would load data from HDF5 result files via `XpcsFile`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.dpi'] = 100

## 1. The XpcsFile API

In a real workflow, you load XPCS data from HDF5 result files:

```python
from xpcsviewer.xpcs_file import XpcsFile

# Load a Multitau result file
xf = XpcsFile("/path/to/A001_Multitau_result.hdf")

# Access metadata
print(f"Analysis type: {xf.atype}")   # e.g. "Multitau"
print(f"Label: {xf.label}")

# Extract G2 data
q_values, t_el, g2, g2_err, labels = xf.get_g2_data()
# Optional filtering:
q_values, t_el, g2, g2_err, labels = xf.get_g2_data(
    qrange=(0.01, 0.05),  # Q range
    trange=(0.001, 100),   # Delay time range in seconds
)

# Extract SAXS 1D profile
q, Iq, xlabel, ylabel = xf.get_saxs1d_data()

# Always close when done (or use as context manager)
xf.close()
```

Below, we generate synthetic data that mimics the output of these methods.

In [ ]:
np.random.seed(42)

# --- Synthetic G2 data (mimics xf.get_g2_data() output) ---
# Physical parameters: diffusive dynamics where tau = 1 / (D * q^2)
D_eff = 5e3           # Effective diffusion coefficient (nm^2/s)
n_q = 10              # Number of Q bins
n_t = 60              # Number of delay points
baseline = 1.0
contrast = 0.25

# Q-values (inverse nanometers, typical XPCS range)
q_values = np.linspace(0.008, 0.04, n_q)
labels = [f"q={q:.4f} nm\u207b\u00b9" for q in q_values]

# Log-spaced delay times (seconds)
t_el = np.logspace(-3, 2, n_t)

# Generate G2 for each Q bin: g2 = baseline + contrast * exp(-2*t/tau)
tau_q = 1.0 / (D_eff * q_values**2)  # tau decreases with Q^2
g2 = np.zeros((n_t, n_q))
g2_err = np.zeros((n_t, n_q))

for i in range(n_q):
    g2_clean = baseline + contrast * np.exp(-2 * t_el / tau_q[i])
    noise = 0.003 + 0.002 * np.random.rand()
    g2[:, i] = g2_clean + np.random.normal(0, noise, size=n_t)
    g2_err[:, i] = noise

# --- Synthetic SAXS 1D (mimics xf.get_saxs1d_data() output) ---
q_saxs = np.logspace(-2, -0.5, 200)
Iq_saxs = 1e4 * q_saxs**(-2.5) * (1 + np.random.normal(0, 0.02, size=q_saxs.shape))
xlabel_saxs = "Q (nm\u207b\u00b9)"
ylabel_saxs = "I(Q) (a.u.)"

print(f"G2 data: {g2.shape[1]} Q bins, {g2.shape[0]} delay points")
print(f"Delay time range: [{t_el.min():.2e}, {t_el.max():.2e}] s")
print(f"Q range: [{q_values.min():.4f}, {q_values.max():.4f}] nm\u207b\u00b9")
print()
print("Relaxation times (tau) from diffusion:")
for i in range(n_q):
    print(f"  {labels[i]:>22s}  tau = {tau_q[i]:.4f} s")

## 2. Plot G2 Curves

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

n_plot = min(6, n_q)
for i in range(n_plot):
    ax.errorbar(
        t_el, g2[:, i], yerr=g2_err[:, i],
        fmt="o", markersize=3, capsize=2,
        label=labels[i],
    )

ax.set_xscale("log")
ax.set_xlabel("Delay time (s)")
ax.set_ylabel(r"$g_2(\tau)$")
ax.set_title("G2 Autocorrelation Functions")
ax.legend(fontsize=8, ncol=2)
plt.tight_layout()
plt.show()

## 3. Q-Range Filtering

Select a subset of Q bins and time range (equivalent to `get_g2_data(qrange=..., trange=...)`).

In [ ]:
# Filter Q range
q_min, q_max = 0.01, 0.03
t_min, t_max = 0.01, 50.0

q_mask = (q_values >= q_min) & (q_values <= q_max)
t_mask = (t_el >= t_min) & (t_el <= t_max)

q_sub = q_values[q_mask]
t_sub = t_el[t_mask]
g2_sub = g2[np.ix_(t_mask, q_mask)]
g2_err_sub = g2_err[np.ix_(t_mask, q_mask)]
labels_sub = [labels[i] for i in range(n_q) if q_mask[i]]

print(f"Filtered: {len(q_sub)} Q bins, {len(t_sub)} time points")

fig, ax = plt.subplots(figsize=(10, 6))
for i in range(len(q_sub)):
    ax.errorbar(t_sub, g2_sub[:, i], yerr=g2_err_sub[:, i],
                fmt="o", markersize=3, capsize=2, label=labels_sub[i])
ax.set_xscale("log")
ax.set_xlabel("Delay time (s)")
ax.set_ylabel(r"$g_2(\tau)$")
ax.set_title(f"Filtered G2: Q=[{q_min}, {q_max}], t=[{t_min}, {t_max}]")
ax.legend(fontsize=8)
plt.tight_layout()
plt.show()

## 4. SAXS 1D Profile

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
ax.loglog(q_saxs, Iq_saxs, "-", linewidth=1)
ax.set_xlabel(xlabel_saxs)
ax.set_ylabel(ylabel_saxs)
ax.set_title("SAXS 1D Profile")
plt.tight_layout()
plt.show()

## 5. G2 Waterfall Plot

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

colors = plt.cm.viridis(np.linspace(0, 1, n_q))
for i in range(n_q):
    ax.semilogx(t_el, g2[:, i], "o-", color=colors[i], markersize=2, linewidth=0.5)

ax.set_xlabel("Delay time (s)")
ax.set_ylabel(r"$g_2(\tau)$")
ax.set_title(f"G2 Overview ({n_q} Q bins)")

sm = plt.cm.ScalarMappable(cmap="viridis", norm=plt.Normalize(q_values.min(), q_values.max()))
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax)
cbar.set_label(r"Q (nm$^{-1}$)")
plt.tight_layout()
plt.show()

## 6. G2 Heatmap

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

im = ax.pcolormesh(
    np.arange(g2.shape[1]),
    t_el,
    g2,
    shading="auto",
    cmap="viridis",
)
ax.set_yscale("log")
ax.set_xlabel("Q bin index")
ax.set_ylabel("Delay time (s)")
ax.set_title("G2 Heatmap")
plt.colorbar(im, ax=ax, label=r"$g_2(\tau)$")
plt.tight_layout()
plt.show()

## 7. Schema-Validated I/O with HDF5Facade

For stricter data integrity, use the `HDF5Facade` for schema-validated reading:

```python
from xpcsviewer.io import HDF5Facade

facade = HDF5Facade()
qmap = facade.read_qmap("/path/to/data.hdf")
g2_data = facade.read_g2_data("/path/to/data.hdf")
```

## Summary

| Task | XpcsFile Method | Returns |
|------|----------------|--------|
| Load file | `XpcsFile(path)` | XpcsFile instance |
| Check type | `xf.atype` | str (e.g. "Multitau") |
| Get metadata | `xf.get_hdf_info()` | dict |
| Extract G2 | `xf.get_g2_data(qrange, trange)` | (q, t, g2, g2_err, labels) |
| Get SAXS | `xf.get_saxs1d_data()` | (q, Iq, xlabel, ylabel) |
| Schema I/O | `HDF5Facade().read_qmap(path)` | QMapSchema |

Next: [02_g2_analysis.ipynb](02_g2_analysis.ipynb) for G2 fitting and analysis.